# 0. Background
1. 一个common sense是，虽然网络变深会利于学习，但是gradient在深度比较深的情况下会接近0
2. 一个22+8层的网络我们希望比22层更好，通过添加short的方式，如果train好了那8层固然好，train不好可以直接走short cut这个路径，不影响前面22层的结果
![](https://gitee.com/wyjyoga/my-pic-go/raw/master/img/202302272145886.png)
3. ![](https://gitee.com/wyjyoga/my-pic-go/raw/master/img/202302272148345.png)

# 1. residual block
1. 一种设计是2个3×的conv+一个short。很显然，**残差块要求2个卷积层的输出与输入形状一样**，才能让x直接加到F(x)上
    - 但是这种设计参数量太大：假设input_channel/ feature map = 256 = output_channel，那么一个conv其实是需要256×256×(3×3)=600K个参数
2. 所以另一种设计是使用1×1大小的conv改变通道数，（先减小再放大）
    - 比如下面代码实现的，只是对x也要改变通道的操作

# 2. Why res？
1. 一种解释是，本身NN学习的是mapping是从x到H(x),现在有了res结构，它把H(x)拆解 = F(x)+x，因此F(x) = H(x) - x，学的是这样一个残差
2. 下图实现的是: ![](https://gitee.com/wyjyoga/my-pic-go/raw/master/img/202302272205010.png)

In [ ]:
from torch import nn
import torch.nn.functional as F
class ResBlk(nn.Module):
    def __init__(self,ch_in,ch_out):
        super(ResBlk, self).__init__()
        self.conv1=nn.Conv2d(ch_in,ch_out,kernel_size=3,stride=1,padding=1)
        self.bn1=nn.BatchNorm2d(ch_out)
        self.conv2=nn.Conv2d(ch_out,ch_out,kernel_size=3,stride=1,padding=1)
        self.bn2=nn.BatchNorm2d(ch_out)

        self.extra=nn.Sequential()
        # 当in=64 out=256，为了使大小一致可以进行element-wise的加法，使用265个1×1大小的kernel
        if ch_out !=ch_in:  # [b,ch_in,h,w]=[b,ch_out,h,w]
            self.extra= nn.Sequential(
            nn.Conv2d(ch_in,ch_out,kernel_size=1,stride=1),
            nn.BatchNorm2d(ch_out))

    def forward(self,x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        out = self.extra(x)+out

# 3. resnet与densenet
1. resnet其实是做了加法，densenet则是做了channel-wise的concat的操作
2. ![](https://gitee.com/wyjyoga/my-pic-go/raw/master/img/202302272216835.png)